In [2]:
from __future__ import print_function
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose, Flatten, Dense, UpSampling2D,Dropout,BatchNormalization,Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
import iou

/var/web_services/galaxy/jupyter_conda/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from numpy.random import seed
seed(13)
from tensorflow import set_random_seed
set_random_seed(22)

In [5]:
print('loading data...')
data_imgs = np.load('imgs_train.npy')
data_masks = np.load('imgs_mask_train.npy')


data_imgs = data_imgs[:,:,:,np.newaxis]
data_masks = data_masks[:,:,:,np.newaxis]

data_imgs = data_imgs.astype('float32')
data_imgs /=255.0

data_masks = data_masks.astype('float32')
data_masks /=255.0





loading data...


In [ ]:
data_masks=to_categorical(data_masks, num_classes=2)

In [6]:
data_masks.shape

(3920, 128, 128, 1)

In [20]:
all_elments= data_masks.shape[0]* data_masks.shape[1]* data_masks.shape[1]* data_masks.shape[3]
nonzeroo = np.count_nonzero(data_masks)
ratio=nonzeroo/all_elments
print(ratio)

0.25301554154376593


In [ ]:
train_imgs, valid_imgs, train_masks, valid_masks = train_test_split(data_imgs, data_masks, test_size=0.2, random_state=54)

In [ ]:
def conv_bn_relu(num_filters, input_features):
    
    output_features = Conv2D(num_filters, (3, 3), padding="same")(input_features)
    output_features = BatchNormalization()(output_features)
    output_features = Activation('relu')(output_features)
    
    output_features = Conv2D(num_filters, (3, 3), padding="same")(output_features)
    output_features = BatchNormalization()(output_features)
    output_features = Activation('relu')(output_features)
    
    return output_features


def uconv_bn_relu(num_filters, input_features, conv_layer):
    
    deconv = Conv2DTranspose(num_filters, (3, 3), strides=(2, 2), padding="same")(input_features)
    uconv = concatenate([deconv, conv_layer])
    uconv = Dropout(0.5)(uconv)
    uconv = Conv2D(num_filters, (3, 3), activation="relu", padding="same")(uconv)
    output_features = Conv2D(num_filters, (3, 3), activation="relu", padding="same")(uconv)
    
    return output_features


def model_arch(input_layer):
    
    start_neurons=16
    
    conv1 = conv_bn_relu(start_neurons * 1,input_layer)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)

    # 64 -> 32
    conv2 = conv_bn_relu(start_neurons * 2,pool1)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    # 32 -> 16
    conv3 = conv_bn_relu(start_neurons * 4,pool2)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)

    # 16 -> 8
    conv4 = conv_bn_relu(start_neurons * 8,pool3)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)

    # Middle
    convm = conv_bn_relu(start_neurons * 4,pool4)

    # 8 -> 16
    uconv4 = uconv_bn_relu(start_neurons * 8, convm, conv4)


    # 16 -> 32
    uconv3 = uconv_bn_relu(start_neurons * 4, uconv4, conv3)

    # 32 -> 64
    uconv2 = uconv_bn_relu(start_neurons * 2, uconv3, conv2)

    # 64 -> 128
    uconv1 = uconv_bn_relu(start_neurons * 1, uconv2, conv1)

    #uconv1 = Dropout(0.5)(uconv1)
    output_layer = Conv2D(2, (1,1), padding="same", activation="softmax")(uconv1)
    
    
    return output_layer

input_layer=Input(shape=(128,128,1))
output_layer=model_arch(input_layer)

model=Model(inputs=input_layer,outputs=output_layer)

In [ ]:
model.summary()

In [4]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=[mean_iou])

NameError: name 'model' is not defined

In [ ]:
early_stopping = EarlyStopping(min_delta=0, patience=20, verbose=1)
model_checkpoint = ModelCheckpoint("checkpoint/keras.model", monitor='val_loss', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [ ]:
batchsize=32
# we create two instances with the same arguments
data_gen_args = dict(horizontal_flip=True,
                     vertical_flip=True)

data_gen_args = dict(horizontal_flip=True,
                     vertical_flip=True)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
seed = 1

image_generator = image_datagen.flow(train_imgs, seed=seed, batch_size=batchsize, shuffle=True)
mask_generator = mask_datagen.flow(train_masks, seed=seed, batch_size=batchsize, shuffle=True)


# Just zip the two generators to get a generator that provides augmented images and masks at the same time
train_generator = zip(image_generator, mask_generator)


# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)

steps_epoch=np.ceil(len(train_imgs)/batchsize)

model.fit_generator(
    train_generator,
    steps_per_epoch=steps_epoch,
    validation_data = (valid_imgs, valid_masks),
    verbose=1,
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    initial_epoch=0,
    epochs=200)

In [7]:
model=Sequential()
model.add(Conv2D(64,(3,3),padding="same",input_shape=(50,50,1)))
model.add(Activation('relu'))
for layers in range(2,6+1):
    model.add(Conv2D(64,(3,3),padding="same"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
model.add(Conv2D(1,(3,3),padding="same"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 50, 50, 64)        640       
_________________________________________________________________
activation_23 (Activation)   (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 50, 50, 64)        36928     
_________________________________________________________________
batch_normalization_21 (Batc (None, 50, 50, 64)        256       
_________________________________________________________________
activation_24 (Activation)   (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 50, 50, 64)        36928     
_________________________________________________________________
batch_normalization_22 (Batc (None, 50, 50, 64)        256       
__________

In [18]:
import numpy as np
preds_test=np.random.rand(50,6,6,2)

i=24
print(preds_test.shape)
print(preds_test[i,:,:,0])
print('tessssssssss')
print(preds_test[i,:,:,1])

preds_test=np.round(preds_test>0.8)
print(preds_test[i,:,:,0])
print('tessssssssss')
print(preds_test[i,:,:,1])


merged_mask=np.zeros(shape=(50,6,6,1),dtype=np.float32)
for ii in range(len(preds_test)):
    ind=np.argmax(preds_test[ii], axis=2)
    ind = ind[:,:,np.newaxis]
    merged_mask[ii]=ind

print('rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr')
print(merged_mask[24,:,:,0])
print(merged_mask.shape)

(50, 6, 6, 2)
[[0.55296593 0.38207445 0.33145426 0.89216627 0.69992017 0.76979006]
 [0.80534255 0.32598075 0.27239627 0.32862343 0.06330973 0.42069691]
 [0.34307432 0.91080512 0.84386402 0.06808622 0.11249593 0.16315418]
 [0.39309258 0.51620931 0.38883916 0.75895259 0.42118214 0.7937163 ]
 [0.2816211  0.0350621  0.5541956  0.10033414 0.27347248 0.35978725]
 [0.23783216 0.95056119 0.28850725 0.29659225 0.88866017 0.5402182 ]]
tessssssssss
[[0.34237716 0.30928394 0.58748592 0.71396289 0.75074721 0.5922782 ]
 [0.96137438 0.11365227 0.5898658  0.63209257 0.43965834 0.59344017]
 [0.30706221 0.34398906 0.34526547 0.28123172 0.01518975 0.61411668]
 [0.58044593 0.9442891  0.98601995 0.4176912  0.28196191 0.80188866]
 [0.06370879 0.2743381  0.38497007 0.99214623 0.46733991 0.12106242]
 [0.22039253 0.39519491 0.48708852 0.21375736 0.67980787 0.94867926]]
[[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0.]]
tessssssssss
[[0

In [21]:
preds_test.shape[1]

6

In [5]:
def conv_bn_relu(num_filters, input_features):
    
    output_features = Conv2D(num_filters, (3, 3), strides=(1,1), padding="same")(input_features)
    output_features = BatchNormalization()(output_features)
    output_features = Activation('relu')(output_features)
    
    output_features = Conv2D(num_filters, (3, 3), padding="same")(output_features)
    output_features = BatchNormalization()(output_features)
    output_features = Activation('relu')(output_features)
    
    return output_features


def uconv_bn_relu(num_filters, input_features, conv_layer):
    
    deconv = Conv2DTranspose(num_filters, (3, 3), strides=(2, 2), padding="same")(input_features)
    uconv = concatenate([deconv, conv_layer])
    uconv = Dropout(0.5)(uconv)
    uconv = Conv2D(num_filters, (3, 3), activation="relu", padding="same")(uconv)
    output_features = Conv2D(num_filters, (3, 3), activation="relu", padding="same")(uconv)
    
    return output_features


def model_arch(input_layer):
    
    start_neurons=16
    
    conv1 = conv_bn_relu(start_neurons * 1,input_layer)
    #pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.25)(conv1)

    # 64 -> 32
    conv2 = conv_bn_relu(start_neurons * 2,pool1)
    #pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.5)(conv2)

    # 32 -> 16
    conv3 = conv_bn_relu(start_neurons * 4,pool2)
    #pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(0.5)(conv3)

    # 16 -> 8
    conv4 = conv_bn_relu(start_neurons * 8,pool3)
    #pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(0.5)(conv4)



    #uconv1 = Dropout(0.5)(uconv1)
    output_layer = Conv2D(1, (1,1), padding="same")(pool4)
    
  
    return output_layer

input_layer=Input(shape=(512,512,1))
output_layer=model_arch(input_layer)

model=Model(inputs=input_layer,outputs=output_layer)


# In[ ]:


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 512, 512, 1)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 512, 512, 16)      160       
_________________________________________________________________
batch_normalization_9 (Batch (None, 512, 512, 16)      64        
_________________________________________________________________
activation_9 (Activation)    (None, 512, 512, 16)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 512, 512, 16)      2320      
_________________________________________________________________
batch_normalization_10 (Batc (None, 512, 512, 16)      64        
_________________________________________________________________
activation_10 (Activation)   (None, 512, 512, 16)      0         
__________